In [1]:
import localgroup
import iso_criteria



import sys
sys.path.append('/u/ki/yymao/pyscripts')
from helpers.SimulationAnalysis import readHlist
from fast3tree import fast3tree

In [98]:
import numpy as np
from sdfpy import load_sdf, SDFIndex, SDFRead
from itertools import product

fname = "/nfs/slac/g/ki/darksky/simulations/ds14_b/ds14_b_halos_1.0000" 
halos_data = SDFRead(fname)
halos_midx = SDFRead(fname + '.midx7')
halos = SDFIndex(halos_data, halos_midx)

fields = 'x y z vx vy vz mvir rvir vmax id mvir_pid'.split()
pad = np.array([2.0, 2.0, 2.0]) #in Mpc/h

def get_data(level, cell_iarr, pad, fields, cut=None, cut_fields=None):
    dtype = np.dtype([(f, int if f.endswith('id') else float) for f in fields])
    count = 0
    for h in halos.iter_padded_bbox_data(level, idx, pad, fields):
        if hasattr(cut, '__call__'):
            flag = cut(*(h[f] for f in cut_fields))
            count_this = np.count_nonzero(flag)
        else:
            flag = slice(None)
            count_this = len(h[fields[0]])
        if count_this:
            if count:
                data.resize(count+count_this)
            else:
                data = np.empty(count_this, dtype)
            for f in fields:
                data[f][count:] = h[f][flag]
            count += count_this
    if not count:
        data = np.empty(0, dtype)
    return data

level = 3
for idx in product(xrange(2**level), repeat=3):
    data = get_data(level, idx, pad, fields, lambda x: x > 40.0, ['vmax'])
    break

In [100]:
data['mvir']

array([  2.21165384e+10,   2.76075418e+10,   1.55578409e+10, ...,
         2.53196227e+10,   1.81508137e+10,   4.09537290e+10])

In [2]:
level = 3
for idx in product(xrange(2**level), repeat=3):
    print idx

NameError: name 'product' is not defined

In [2]:
#path = '/nfs/slac/g/ki/ki20/cosmo/behroozi/Consuelo/4001/hlists/hlist_1.00000.list'
#halos = readHlist(path)

In [3]:
#np.save('/lustre/ki/pfs/mwillia1/LG_project/Consuelo_Boxes/4001hlist.npy', halos)

In [9]:
halos = np.load('/lustre/ki/pfs/mwillia1/LG_project/Consuelo_Boxes/4001hlist.npy')

In [16]:
halos = halos[halos['vmax']>100]

In [17]:
halos.shape

(1290811,)

In [19]:
host_flag = np.where(halos['upid'] == -1)[0]

In [40]:
def find_pairs(halos, host_flag, D):
    pairs = []
    points = halos[host_flag][list('xyz')].view(float).reshape(-1, 3)
    with fast3tree(points) as tree:
        for i, p in enumerate(points):
            idx = tree.query_radius(p, D, True, 'index')
            idx = idx[idx > i]
            pairs.extend(((i, j) if halos['mvir'][host_flag[i]] > halos['mvir'][host_flag[j]] else (j, i) for j in idx))
    return pairs

def find_periodic_midpoint(p1, p2, box_size, periodic=True):
    mid = (p1 + p2)*0.5
    if not periodic:
        return mid
    half_box_size = box_size*0.5
    mid[np.fabs(p1 - p2) > half_box_size] += half_box_size
    mid = np.fmod(mid, box_size)
    return mid

def isolated(pairs, halos, host_flag, D_iso, D_M33, box_size, periodic=True):
    print 'before: ', len(pairs)
#    iso_pairs = []
    MW_M31_larger = []
    M31_M31_larger = []
    M33_M31_larger = []
    M33_M33_larger = []
    M31_M33_larger = []
    MW_M33_larger = []
    with fast3tree(halos[list('xyz')].view(float).reshape(-1, 3)) as tree:
        for pair in pairs:
            print pair
            h1, h2 = halos[host_flag[list(pair)]] #h1 is the larger halo
            p1 = np.fromiter((h1[ax] for ax in 'xyz'), float)
            p2 = np.fromiter((h2[ax] for ax in 'xyz'), float)

            mid = find_periodic_midpoint(p1, p2, box_size, periodic)
            idx = tree.query_radius(mid, D_iso, periodic, 'index')

            biggest = halos['mvir'][idx].argmax()
            idx = np.delete(idx, biggest)
            biggest = halos['mvir'][idx].argmax() #biggest is now second-biggest
            

            # the second index has a smaller mass
            if h2['mvir'] == halos['mvir'][idx[biggest]]:
#                iso_pairs.append(pair) #satisfying iso
                M31_M31_larger.append(pair[0])
                MW_M31_larger.append(pair[1])
            else:
                continue #no need to search M33
            
                

            #find M33, in two ways
            idx = tree.query_radius(p1, D_M33, periodic, 'index')
#            idx = idx[halos['vmax'][idx] < 80]
#            M33_ind = idx[halos['mvir'][idx].argmax()] if len(idx) else -1
            m31_delete_ind = halos['mvir'][idx].argmax()
            idx = np.delete(idx, m31_delete_ind) #So we don't identify M31 as M33
            if len(idx):
                m33_candidate = halos['mvir'][idx].argmax()
                if halos[idx[m33_candidate]]['id']==halos[host_flag][pair[1]]['id']: #If MW is identified as M33
                    idx = np.delete(idx, m33_candidate)
            M33_ind = idx[halos['mvir'][idx].argmax()] if len(idx) else -1
            if halos['vmax'][M33_ind] > 80: M33_ind = -1
            M33_M31_larger.append(M33_ind)

            idx = tree.query_radius(p2, D_M33, periodic, 'index')
#            idx = idx[halos['vmax'][idx] < 80]
#            LMC_ind = idx[halos['mvir'][idx].argmax()] if len(idx) else -1
            m31_delete_ind = halos['mvir'][idx].argmax()
            idx = np.delete(idx, m31_delete_ind) #So we don't identify M31 as M33
            if len(idx):
                m33_candidate = halos['mvir'][idx].argmax()
                if halos[idx[m33_candidate]]['id']==halos[host_flag][pair[1]]['id']: #If MW is identified as M33
                    idx = np.delete(idx, m33_candidate)
            LMC_ind = idx[halos['mvir'][idx].argmax()] if len(idx) else -1
            if halos['vmax'][LMC_ind] > 80: LMC_ind = -1
                
            M33_mass = halos['mvir'][M33_ind] if M33_ind != -1 else 0
            LMC_mass = halos['mvir'][LMC_ind] if LMC_ind != -1 else 0

            if LMC_mass > M33_mass:
                M33_ind = LMC_ind
                M31_M33_larger.append(pair[1]) #append the smaller mass index in the pair -> M31 since LMC > M33
                MW_M33_larger.append(pair[0]) #append the larger mass index in the pair -> MW since LMC > M33
            else:
                M31_M33_larger.append(pair[0]) #append the smaller mass index in the pair -> M31 since LMC < M33
                MW_M33_larger.append(pair[1]) #append the larger mass index in the pair -> MW since LMC < M33
            M33_M33_larger.append(M33_ind)

    print 'after: ', len(MW_M31_larger)
    return MW_M31_larger, M31_M31_larger, M33_M31_larger, M33_M33_larger, M31_M33_larger, MW_M33_larger


In [20]:
pairs = find_pairs(halos, host_flag, 1.0)

In [38]:
len(pairs)

416667

In [41]:
MW_M31_larger, M31_M31_larger, M33_M31_larger, M33_M33_larger, M31_M33_larger, MW_M33_larger = isolated(pairs, halos, host_flag, 2.0/0.7, 0.4/0.7, 420, periodic=True)

before:  416667
(25, 104012)
(25, 368863)
(26, 192790)
(26, 212589)
(27, 503751)
(30, 104088)
(30, 172252)
(32, 596240)
(32, 467)
(35, 315434)
(35, 28824)
(36, 273447)
(36, 398019)
(37, 74992)
(38, 142448)
(38, 388189)
(38, 74443)
(39, 315803)
(42, 473353)
(42, 398053)
(42, 347468)
(43, 208)
(43, 558217)
(43, 47862)
(44, 283957)
(44, 326610)
(44, 254361)
(45, 115492)
(45, 56229)
(45, 315796)
(47, 397853)
(48, 415779)
(52, 398081)
(54, 74413)
(55, 473293)
(55, 254113)
(55, 305760)
(56, 572125)
(57, 517597)
(58, 273134)
(58, 94769)
(58, 377287)
(59, 47282)
(63, 47491)
(63, 273182)
(66, 201971)
(67, 48008)
(68, 338510)
(72, 20602)
(72, 449762)
(72, 296649)
(73, 192353)
(74, 473)
(75, 115312)
(75, 434070)
(75, 212570)
(75, 606146)
(76, 434208)
(76, 142904)
(76, 182973)
(77, 627205)
(77, 368710)
(77, 442279)
(78, 467668)
(79, 576056)
(79, 273859)
(80, 623186)
(80, 153870)
(81, 425285)
(81, 415845)
(81, 434207)
(81, 306197)
(81, 489866)
(82, 377156)
(82, 338257)
(83, 56940)
(83, 434169)
(84,

KeyboardInterrupt: 

In [22]:
a=np.array(M33_M31_larger)
len(a[a[:] > 0])

54241

In [88]:
#hosts_subs = halos[halos['vmax']>80]
#zall = np.vstack((hosts_subs['x'], hosts_subs['y'], hosts_subs['z'])).T
#zall = zall[1100000:1200000]
#masses_subs = hosts_subs['mvir'][1100000:1200000]

In [5]:
#hosts = halos[halos['upid']==-1]
#hosts = hosts[hosts['vmax']>80]
#hosts.shape

(1876946,)

In [6]:
#z = np.vstack((hosts['x'], hosts['y'], hosts['z'])).T
#z = z[1100000:1200000]
#masses = hosts['mvir'][1100000:1200000]

In [12]:
#for i in range(z.shape[1]):
#    print z[:,i].max()

419.99866
419.99811
419.99713


In [59]:
#pairs = iso_criteria.criteria.find_pairs(z, 1.0)

In [60]:
len(pairs)

13347

In [13]:
isopairs = iso_criteria.criteria.isolated(pairs, 2, z, masses, 420, True)

before:  13347
after:  5484


In [84]:
MW_inds, M31_inds = iso_criteria.criteria.label_MW_M31(isopairs, masses)

In [90]:
M33_inds = iso_criteria.criteria.find_M33(zall, 0.4, masses_subs, M31_inds, z, True)


In [94]:
M33_inds = np.array(M33_inds)
a=np.array(M33_inds)[M33_inds>0]

In [104]:
len(M33_inds),len(M31_inds),len(MW_inds)

(5484, 5484, 5484)

In [82]:
def get_data(halos, host_flag, MW_inds, M31_inds, M33_inds):
    fn = ['MW_'+nm for nm in halos.dtype.names]
    fn1 = ['M31_'+nm for nm in halos.dtype.names]
    fn2 = ['M33_'+nm for nm in halos.dtype.names]
    fields = np.hstack((np.array(fn), np.array(fn1), np.array(fn2)))
    dtype = np.dtype([(f, int if f.endswith('id') else float) for f in fields])
    num_sys = len(MW_inds)
    data = np.empty(num_sys, dtype)
    no_M33_data = np.copy(halos[0])
    for f in no_M33_data.dtype.names: no_M33_data[f]=-1
    mw_dat = halos[host_flag[MW_M31_larger]]
    m31_dat = halos[host_flag[M31_M31_larger]]
    m33_dat = np.array([halos[i] if i>0 else no_M33_data for i in M33_M31_larger])
    for i in range(len(data)):
        data[i] = tuple(mw_dat[i])+tuple(m31_dat[i])+tuple(m33_dat[i])
    return data

In [37]:
mw_dat = halos[host_flag[MW_M31_larger]]
m31_dat = halos[host_flag[M31_M31_larger]]
m33_dat = halos[M33_M31_larger]

In [65]:
no_M33_data = np.copy(halos[0])
for f in no_M33_data.dtype.names: no_M33_data[f]=-1
no_M33_data

array((-1.0, -1, -1.0, -1, -1, -1, -1, -1, -1, -1.0, -1.0, -1.0, -1.0, -1.0, -1, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1, -1, -1, -1, -1, -1, -1, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0), 
      dtype=[('scale', '<f8'), ('id', '<i8'), ('desc_scale', '<f8'), ('desc_id', '<i8'), ('num_prog', '<i8'), ('pid', '<i8'), ('upid', '<i8'), ('desc_pid', '<i8'), ('phantom', '<i8'), ('sam_mvir', '<f8'), ('mvir', '<f8'), ('rvir', '<f8'), ('rs', '<f8'), ('vrms', '<f8'), ('mmp?', '<i8'), ('scale_of_last_MM', '<f8'), ('vmax', '<f8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<f8'), ('Jx', '<f8'), ('Jy', '<f8'), ('Jz', '<f8'), ('Spin', '<f8'), ('Breadth_first_ID', '<i8'), ('Depth_first_ID', '<i8'), ('Tree_root_ID', '<i8'), ('Orig_halo_ID', '<i8'), ('Snap_num', '<i8'), ('Next_coprogenitor_depthfirst_ID', '<i8'), ('Last_progenitor_depthfirst_ID', '<i8'), ('Macc', '<f8'), ('Mpeak', '<f8'), ('Vacc', '<f8'), ('Vpeak', '<f8'), ('H

In [77]:
m33_dat = np.array([halos[i] if i>0 else no_M33_data for i in M33_M31_larger])

In [39]:
fn = ['MW_'+nm for nm in halos.dtype.names]
fn1 = ['M31_'+nm for nm in halos.dtype.names]
fn2 = ['M33_'+nm for nm in halos.dtype.names]
fields = np.hstack((np.array(fn), np.array(fn1), np.array(fn2)))
dtype = np.dtype([(f, int if f.endswith('id') else float) for f in fields])
num_sys = len(MW_M31_larger)
data = np.empty(num_sys, dtype)

In [78]:
for i in range(len(data)):
    data[i] = tuple(mw_dat[i])+tuple(m31_dat[i])+tuple(m33_dat[i])

In [81]:
data['M31_id']

array([606844252, 606957793, 605311460, ..., 609735815, 607990729,
       607991489])

In [83]:
dat = get_data(halos, host_flag, MW_M31_larger, M31_M31_larger, M33_M31_larger)

In [96]:
t = np.copy(dat[0:3])
u = np.copy(dat[0:3])
t

array([ (1.0, 606844259, 0.0, -1, 1.0, -1, -1, -1, 0.0, 26653300000000.0, 13350000000000.0, 495.849, 54.025, 401.67, 1.0, 0.5773, 407.03, 221.37297, 96.45479, 282.2254, 158.65, -28.63, -418.32, -2746000000000.0, -87580000000000.0, 71040000000000.0, 0.07386, 41335940.0, 41335940.0, 606844259.0, 1008523.0, 118.0, 41335939.0, 41337338.0, 13350000000000.0, 26920000000000.0, 407.03, 512.64, 0.5624, -769.4, -739.5, -1005.0, 1.0, 606844252, 0.0, -1, 1.0, -1, -1, -1, 0.0, 84630000000000.0, 84630000000000.0, 917.649, 124.256, 731.36, 1.0, 0.3162, 713.08, 221.76477, 95.77451, 281.66891, 459.36, -7.69, 133.55, -1547000000000000.0, -3144000000000000.0, 351800000000000.0, 0.11874, 12229660.0, 12229660.0, 606844252.0, 1008516.0, 118.0, 12229659.0, 12234142.0, 84630000000000.0, 84630000000000.0, 713.08, 718.86, 0.4682, 6045.0, 6122.0, 2528.0, -1.0, -1, -1.0, -1, -1.0, -1, -1, -1, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1

In [97]:
np.random.shuffle(u)
u

array([ (1.0, 606957801, 0.0, -1, 1.0, -1, -1, -1, 0.0, 2155000000000.0, 2102000000000.0, 267.739, 23.576, 221.53, 1.0, 0.2848, 229.62, 267.07373, 65.61007, 410.64505, 178.24, -127.55, 35.75, -4367000000000.0, -98990000000.0, 1732000000000.0, 0.0653, 89610536.0, 89610536.0, 606957801.0, 278525.0, 118.0, 89610535.0, 89610769.0, 2102000000000.0, 2158000000000.0, 229.62, 256.53, 0.3421, 65.95, 56.05, -21.89, 1.0, 606957793, 0.0, -1, 2.0, -1, -1, -1, 0.0, 76436700000000.0, 76300000000000.0, 886.498, 116.699, 713.78, 1.0, 0.6083, 692.6, 267.50497, 64.81881, 411.01257, 309.17, 448.6, 401.55, 188600000000000.0, 717900000000000.0, 1992000000000000.0, 0.07235, 13383055.0, 13383055.0, 606957793.0, 278517.0, 118.0, 13383054.0, 13388081.0, 76300000000000.0, 76510000000000.0, 692.6, 759.97, 0.5926, -2198.0, -2010.0, 2525.0, -1.0, -1, -1.0, -1, -1.0, -1, -1, -1, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -

In [199]:
M33_inds.index(977)

68

In [202]:
hosts[4568]

(1.0, 604562582, 0.0, -1, 1, -1, -1, -1, 0, 9712000000000.0, 9712000000000.0, 445.932, 106.416, 342.01, 1, 0.3421, 318.06, 13.88385, 352.27481, 25.54296, -368.93, -245.38, -22.94, 73350000000000.0, -134400000000000.0, -38090000000000.0, 0.15966, 48281764, 48281764, 604562582, 265336, 118, 48281763, 48282186, 9712000000000.0, 9712000000000.0, 318.06, 341.4, 0.5926, 681.4, 700.9, 1162.0)

In [207]:
dat[0]['MWid']

607898845

In [118]:
-1*np.ones(11)

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [125]:
hosts[0]

(1.0, 606949868, 0.0, -1, 7, -1, -1, -1, 0, 519300000000000.0, 519300000000000.0, 1679.949, 325.19, 1340.86, 1, 0.5478, 1230.37, 215.5239, 67.77184, 395.20154, 139.05, 214.51, 267.84, -1.262e+16, 6145000000000000.0, 1.634e+16, 0.028, 1119673, 1119673, 606949868, 270592, 118, 1119672, 1137446, 519300000000000.0, 519300000000000.0, 1230.37, 1373.4, 0.5773, 60450.0, 60920.0, 58610.0)

In [136]:
hosts.dtype

dtype([('scale', '<f8'), ('id', '<i8'), ('desc_scale', '<f8'), ('desc_id', '<i8'), ('num_prog', '<i8'), ('pid', '<i8'), ('upid', '<i8'), ('desc_pid', '<i8'), ('phantom', '<i8'), ('sam_mvir', '<f8'), ('mvir', '<f8'), ('rvir', '<f8'), ('rs', '<f8'), ('vrms', '<f8'), ('mmp?', '<i8'), ('scale_of_last_MM', '<f8'), ('vmax', '<f8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<f8'), ('Jx', '<f8'), ('Jy', '<f8'), ('Jz', '<f8'), ('Spin', '<f8'), ('Breadth_first_ID', '<i8'), ('Depth_first_ID', '<i8'), ('Tree_root_ID', '<i8'), ('Orig_halo_ID', '<i8'), ('Snap_num', '<i8'), ('Next_coprogenitor_depthfirst_ID', '<i8'), ('Last_progenitor_depthfirst_ID', '<i8'), ('Macc', '<f8'), ('Mpeak', '<f8'), ('Vacc', '<f8'), ('Vpeak', '<f8'), ('Halfmass_Scale', '<f8'), ('Acc_Rate_Inst', '<f8'), ('Acc_Rate_100Myr', '<f8'), ('Acc_Rate_Tdyn', '<f8')])

In [141]:
hosts.dtype.names

('scale',
 'id',
 'desc_scale',
 'desc_id',
 'num_prog',
 'pid',
 'upid',
 'desc_pid',
 'phantom',
 'sam_mvir',
 'mvir',
 'rvir',
 'rs',
 'vrms',
 'mmp?',
 'scale_of_last_MM',
 'vmax',
 'x',
 'y',
 'z',
 'vx',
 'vy',
 'vz',
 'Jx',
 'Jy',
 'Jz',
 'Spin',
 'Breadth_first_ID',
 'Depth_first_ID',
 'Tree_root_ID',
 'Orig_halo_ID',
 'Snap_num',
 'Next_coprogenitor_depthfirst_ID',
 'Last_progenitor_depthfirst_ID',
 'Macc',
 'Mpeak',
 'Vacc',
 'Vpeak',
 'Halfmass_Scale',
 'Acc_Rate_Inst',
 'Acc_Rate_100Myr',
 'Acc_Rate_Tdyn')

In [186]:
fn = ['MW'+nm for nm in hosts.dtype.names]
fn1 = ['M31'+nm for nm in hosts.dtype.names]
fn2 = ['M33'+nm for nm in hosts.dtype.names]

In [188]:
np.hstack((np.array(fn), np.array(fn1), np.array(fn2)))

array(['MWscale', 'MWid', 'MWdesc_scale', 'MWdesc_id', 'MWnum_prog',
       'MWpid', 'MWupid', 'MWdesc_pid', 'MWphantom', 'MWsam_mvir',
       'MWmvir', 'MWrvir', 'MWrs', 'MWvrms', 'MWmmp?',
       'MWscale_of_last_MM', 'MWvmax', 'MWx', 'MWy', 'MWz', 'MWvx', 'MWvy',
       'MWvz', 'MWJx', 'MWJy', 'MWJz', 'MWSpin', 'MWBreadth_first_ID',
       'MWDepth_first_ID', 'MWTree_root_ID', 'MWOrig_halo_ID',
       'MWSnap_num', 'MWNext_coprogenitor_depthfirst_ID',
       'MWLast_progenitor_depthfirst_ID', 'MWMacc', 'MWMpeak', 'MWVacc',
       'MWVpeak', 'MWHalfmass_Scale', 'MWAcc_Rate_Inst',
       'MWAcc_Rate_100Myr', 'MWAcc_Rate_Tdyn', 'M31scale', 'M31id',
       'M31desc_scale', 'M31desc_id', 'M31num_prog', 'M31pid', 'M31upid',
       'M31desc_pid', 'M31phantom', 'M31sam_mvir', 'M31mvir', 'M31rvir',
       'M31rs', 'M31vrms', 'M31mmp?', 'M31scale_of_last_MM', 'M31vmax',
       'M31x', 'M31y', 'M31z', 'M31vx', 'M31vy', 'M31vz', 'M31Jx', 'M31Jy',
       'M31Jz', 'M31Spin', 'M31Breadth_first_ID'

In [185]:
np.array(fn).flatten()

array(['MWscale', 'MWid', 'MWdesc_scale', 'MWdesc_id', 'MWnum_prog',
       'MWpid', 'MWupid', 'MWdesc_pid', 'MWphantom', 'MWsam_mvir',
       'MWmvir', 'MWrvir', 'MWrs', 'MWvrms', 'MWmmp?',
       'MWscale_of_last_MM', 'MWvmax', 'MWx', 'MWy', 'MWz', 'MWvx', 'MWvy',
       'MWvz', 'MWJx', 'MWJy', 'MWJz', 'MWSpin', 'MWBreadth_first_ID',
       'MWDepth_first_ID', 'MWTree_root_ID', 'MWOrig_halo_ID',
       'MWSnap_num', 'MWNext_coprogenitor_depthfirst_ID',
       'MWLast_progenitor_depthfirst_ID', 'MWMacc', 'MWMpeak', 'MWVacc',
       'MWVpeak', 'MWHalfmass_Scale', 'MWAcc_Rate_Inst',
       'MWAcc_Rate_100Myr', 'MWAcc_Rate_Tdyn'], 
      dtype='|S33')

In [162]:
t = np.copy(hosts[0])

In [169]:
for f in t.dtype.names: t[f]=-1

In [171]:
t

array((-1.0, -1, -1.0, -1, -1, -1, -1, -1, -1, -1.0, -1.0, -1.0, -1.0, -1.0, -1, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1, -1, -1, -1, -1, -1, -1, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0), 
      dtype=[('scale', '<f8'), ('id', '<i8'), ('desc_scale', '<f8'), ('desc_id', '<i8'), ('num_prog', '<i8'), ('pid', '<i8'), ('upid', '<i8'), ('desc_pid', '<i8'), ('phantom', '<i8'), ('sam_mvir', '<f8'), ('mvir', '<f8'), ('rvir', '<f8'), ('rs', '<f8'), ('vrms', '<f8'), ('mmp?', '<i8'), ('scale_of_last_MM', '<f8'), ('vmax', '<f8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<f8'), ('Jx', '<f8'), ('Jy', '<f8'), ('Jz', '<f8'), ('Spin', '<f8'), ('Breadth_first_ID', '<i8'), ('Depth_first_ID', '<i8'), ('Tree_root_ID', '<i8'), ('Orig_halo_ID', '<i8'), ('Snap_num', '<i8'), ('Next_coprogenitor_depthfirst_ID', '<i8'), ('Last_progenitor_depthfirst_ID', '<i8'), ('Macc', '<f8'), ('Mpeak', '<f8'), ('Vacc', '<f8'), ('Vpeak', '<f8'), ('H

In [208]:
def sq(x):
    return x**2

In [209]:
sq(3.0/2)

2.25

In [6]:
a = np.array([1,2,3,4,5,6])

In [8]:
a

array([1, 2, 4, 5, 6])

In [15]:
halos[host_flag][0]['id']

606949868